# Interacting with Campaigns and Filters <a class="anchor" id="top"></a>

Now that Unicorn Post has trained models for 2 different use cases (User Personalization and Personalized Reranking), we need to integrate them into our application. Amazon Personalize can make recommendations available via an Application Programming Interface (API). In addition, Amazon Personalize includes features that allow you to easily integrate into applications and provide benefits like real time vending of recommendations based on recent application activity.

In this notebook, you will interact with campaigns and filters you created earlier in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`01_Data_Layer.ipynb`](01_Data_Layer.ipynb), and [`02_Training_Layer.ipynb`](02_Training_Layer.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions, and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have two deployed Campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

In this Notebook we will interact with the Campaigns and get recommendations. 

We will interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

The following diagram shows the resources that we will create in this section. The part we are building in this notebook highlighted in blue with a dashed outline.

![Workflow](Images/03_Inference_Layer_Resources.jpg)

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [2]:
#retrieves previously stored variables 
%store -r 

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with Campaigns <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the models have been trained, lets have a look at the recommendations we can get for our users!

### User Personalization Model

"User Personalization " requires a user as input, and it will return the items it thinks the customer is most likely to interact with next.

The cells below will handle getting recommendations from the "User Personalization Model" and rendering the results. Let's see what the recommendations are for a user.

We will be using the `campaignArn`, the `userId`, as well as the number or results we want, `numResults`.

### Select a User

We'll just pick a random user for simplicity. Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

#### Sample User ID's
 -8845298781299428018
 -1032019229384696495
 -1130272294246983140
 344280948527967603
 -445337111692715325

In [4]:
sample_user = str(-8845298781299428018)

In [5]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 5
)

In [6]:
print(get_recommendations_response)

{'ResponseMetadata': {'RequestId': '58860ecf-e08d-47c3-b0fd-1a0980615f11', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 29 May 2024 01:27:47 GMT', 'content-type': 'application/json', 'content-length': '681', 'connection': 'keep-alive', 'x-amzn-requestid': '58860ecf-e08d-47c3-b0fd-1a0980615f11'}, 'RetryAttempts': 0}, 'itemList': [{'itemId': '3566197569262766169', 'score': 0.956582}, {'itemId': '-6623581327558800021', 'score': 0.0052045}, {'itemId': '717471739656838182', 'score': 0.0028103}, {'itemId': '-1415040208471067980', 'score': 0.0016436}, {'itemId': '8482750322470893687', 'score': 0.0015941}], 'recommendationId': 'RID-dc-4d6f-8588-d89f6936024d-CID-482b34'}


A little hard to read - lets make a dataframe

In [7]:
recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])

In [8]:
recommendations_df

,itemId,score
0,3566197569262766169,0.956582
1,-6623581327558800021,0.005204
2,717471739656838182,0.002810
3,-1415040208471067980,0.001644
4,8482750322470893687,0.001594


Lets lookup what sort of articles are being recommended to this user

In [9]:
articles_mlfeatures['item_id'] = articles_mlfeatures['item_id'].astype('str')

In [10]:
recommendations_df = recommendations_df.merge(articles_mlfeatures, how='left', left_on='itemId', right_on='item_id')

In [11]:
recommendations_df

,itemId,score,creation_timestamp,item_id,lang,article_genre,training_text
0,3566197569262766169,0.956582,1486520793,3566197569262766169,en,cloud provider news,Google extends Gmail API for more granular ema...
1,-6623581327558800021,0.005204,1487157951,-6623581327558800021,en,cloud provider news,Google's Spanner database offers global replic...
2,717471739656838182,0.002810,1486633628,717471739656838182,en,cloud provider news,Evernote migrates from own data centers to Goo...
3,-1415040208471067980,0.001644,1482409156,-1415040208471067980,en,cloud provider news,Google Cloud Platform releases icon library fo...
4,8482750322470893687,0.001594,1471309031,8482750322470893687,en,cloud provider news,Google consolidates Hangouts On Air into YouTu...


In [12]:
for i, row in recommendations_df.iterrows():
    print(row['training_text'])
    print("===================================")

Google extends Gmail API for more granular email settings management Google updated the Gmail API with new endpoints to manage filters, aliases, forwarding, signatures, vacation responders, and other granular email settings. This replaces the deprecated Email Settings API. Google has extended the Gmail API with new endpoints for managing email settings like filters, forwarding addresses, IMAP/POP settings, sendas aliases, signatures, and vacation responders. Developers can now retrieve and update signatures for sendas aliases, configure forwarding to external addresses, configure sendas aliases through external providers, use HTML in vacation messages, and manipulate settings for gmail.com accounts. More settings features like mailbox delegate support will be added over time. Most settings endpoints work for any Google Apps or Gmail account, but sensitive operations like modifying aliases or forwarding are restricted to service accounts with domainwide authority. The existing Email Set

What has this user viewed previously

In [13]:
viewed_interactions = interaction_data[interaction_data['user_id'].astype(str) == sample_user].sort_values('timestamp', ascending=False)

In [14]:
viewed_interactions

,timestamp,event_type,item_id,user_id,session_id,user_device_type
69083,1487360168,FOLLOW,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69085,1487360167,COMMENT CREATED,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69082,1487359849,VIEW,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69722,1487069095,VIEW,-8900113512825364282,-8845298781299428018,8663979798581613597,NonMobile
68453,1485973216,VIEW,-532999578436827210,-8845298781299428018,-5430065457414428568,NonMobile
...,...,...,...,...,...,...
12169,1459345020,VIEW,7973573994178035769,-8845298781299428018,3760091107461406486,NonMobile
18783,1459285852,VIEW,6152652267138213180,-8845298781299428018,-6283148774987755959,NonMobile
692,1459274282,VIEW,-1672166631728511207,-8845298781299428018,-6283148774987755959,NonMobile
684,1459274266,VIEW,-1672166631728511207,-8845298781299428018,-6283148774987755959,NonMobile


lets take the most recent 5 articles interacted with by this user

In [15]:
most_recent_five_articles = viewed_interactions.item_id.unique()[:5].astype(str).tolist()

In [16]:
most_recent_five_articles_metadata = articles_mlfeatures[articles_mlfeatures['item_id'].isin(most_recent_five_articles)]

In [17]:
most_recent_five_articles_metadata

,creation_timestamp,item_id,lang,article_genre,training_text
2869,1481676932,8526042588044002101,en,tech,"Cloud Native enables efficient, automated infr..."
2982,1484838924,7419040071212162906,en,tech,"Google search traffic hack by targeting \""best..."
3040,1485899108,-532999578436827210,en,cloud provider news,IBM launches cloud graph database service usin...
3065,1486520793,3566197569262766169,en,cloud provider news,Google extends Gmail API for more granular ema...
3084,1486999803,-8900113512825364282,en,non tech,Report finds banks lag in customer experience ...


We see that the user has previously red information on cloud applications and google in particular. In fact one of our recommendations is actually in the users recent interaction history. This is not ideal lets use the fitler we created earlier to exclude this data. Also lets make use of the return metadata function to retrieve the information on this article so that the recommendations are easier for us to understand. 

In [18]:
response = personalize.update_campaign(
    campaignArn = workshop_userpersonalization_campaign_arn,
    campaignConfig={
        'enableMetadataWithRecommendations': True
    }
)
print (response)

response = personalize.update_campaign(
    campaignArn = workshop_rerank_campaign_arn,
    campaignConfig={
        'enableMetadataWithRecommendations': True
    }
)
print (response)

{'campaignArn': 'arn:aws:personalize:us-east-1:381491864570:campaign/immersion_day_user_personalization_news_campaign', 'ResponseMetadata': {'RequestId': 'ca657b1e-56ea-4eef-94d5-8c05869c876b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 29 May 2024 01:27:51 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '118', 'connection': 'keep-alive', 'x-amzn-requestid': 'ca657b1e-56ea-4eef-94d5-8c05869c876b', 'strict-transport-security': 'max-age=47304000; includeSubDomains', 'x-frame-options': 'DENY', 'cache-control': 'no-cache', 'x-content-type-options': 'nosniff'}, 'RetryAttempts': 0}}
{'campaignArn': 'arn:aws:personalize:us-east-1:381491864570:campaign/immersion_day_personalized_ranking_news_campaign', 'ResponseMetadata': {'RequestId': '50ab22c2-9b77-4d28-8b7a-11e815265bf1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 29 May 2024 01:27:51 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '118', 'connection': 'keep-alive', 'x-amzn-req

In [19]:
up_version_response

NameError: name 'up_version_response' is not defined

In [ ]:
max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
        
    # Personalized ranking Campaign
    up_version_response = personalize.describe_campaign(
        campaignArn = workshop_userpersonalization_campaign_arn
    )

    up_status_campaign_update = up_version_response['campaign']['latestCampaignUpdate']['status']

    if up_status_campaign_update == 'ACTIVE':
        print('Update succeeded for {}'.format(workshop_userpersonalization_campaign_arn))
    elif up_status_campaign_update == "CREATE FAILED":
        print('Update failed for {}'.format(workshop_userpersonalization_campaign_arn))
        break
    
    
    if not up_status_campaign_update == "ACTIVE":
        print('The campaign update is still in progress')
    else:
        print("The Personalized Ranking Campaign is ACTIVE")

        
    pr_version_response = personalize.describe_campaign(
        campaignArn = workshop_rerank_campaign_arn
    )

    pr_status_campaign_update = pr_version_response['campaign']['latestCampaignUpdate']['status']

    if pr_status_campaign_update == 'ACTIVE':
        print('Update succeeded for {}'.format(workshop_rerank_campaign_arn))
    elif pr_status_campaign_update == "CREATE FAILED":
        print('Update failed for {}'.format(workshop_rerank_campaign_arn))
        break
    
    
    if not pr_status_campaign_update == "ACTIVE":
        print('The campaign update is still in progress')
    else:
        print("The Personalized Ranking Campaign is ACTIVE")
        
    if up_status_campaign_update == "ACTIVE" and pr_status_campaign_update == 'ACTIVE':
        break
    
    sleep(60)
    print()

Note you can enable metadata with recommendations when you deploy the campaign for the first time as follows:

```python
user_personalization_create_campaign_response = personalize.create_campaign(
    name = workshop_userpersonalization_campaign_name,
    solutionVersionArn = workshop_userpersonalization_solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {"enableMetadataWithRecommendations": True}    
)
```

In [20]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 10,
    metadataColumns = {"ITEMS": ["training_text"]}
)

In [21]:
get_recommendations_response

{'ResponseMetadata': {'RequestId': '4aa467be-d862-4a04-8038-44c3e8e22b9d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 29 May 2024 01:27:52 GMT',
   'content-type': 'application/json',
   'content-length': '13494',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4aa467be-d862-4a04-8038-44c3e8e22b9d'},
  'RetryAttempts': 0},
 'itemList': [{'itemId': '3566197569262766169',
   'score': 0.956582,
   'metadata': {'training_text': 'Google extends Gmail API for more granular email settings management Google updated the Gmail API with new endpoints to manage filters, aliases, forwarding, signatures, vacation responders, and other granular email settings. This replaces the deprecated Email Settings API. Google has extended the Gmail API with new endpoints for managing email settings like filters, forwarding addresses, IMAP/POP settings, sendas aliases, signatures, and vacation responders. Developers can now retrieve and update signatures for sendas aliases, configure forward

To make processing this easier lets make a helper function which retrieves the interaction history for our users.

In [23]:
response = personalize_events.put_events(
    trackingId="4ea6be84-eb39-4b11-8f59-b76609ac56df",
    userId='madeup-user',
    sessionId='string',
    eventList=[
        {
            'eventId': 'madeupevent111',
            'eventType': 'VIEW',
            'itemId': 'madeupitem111',
            'sentAt': 1714006143,
        },
    ]
)

In [24]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 5,
    filterArn = "arn:aws:personalize:us-east-1:381491864570:filter/timestamp-filter-test",
    filterValues = {"CUTOFF": "\"1614000000\""}
)

In [25]:
get_recommendations_response

{'ResponseMetadata': {'RequestId': '10ed9cd5-6722-4b13-b05e-6b19e457f7ef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 29 May 2024 01:27:57 GMT',
   'content-type': 'application/json',
   'content-length': '185',
   'connection': 'keep-alive',
   'x-amzn-requestid': '10ed9cd5-6722-4b13-b05e-6b19e457f7ef'},
  'RetryAttempts': 0},
 'itemList': [{'itemId': 'madeupitem222', 'score': 1.0}],
 'recommendationId': 'RID-72-41be-b0c9-a41460408fbb-CID-482b34'}

In [26]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = 'madeup-user333',
    numResults = 5
)

In [27]:
get_recommendations_response

{'ResponseMetadata': {'RequestId': '2a3ee44f-522c-4838-b015-6fb1057a20f1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 29 May 2024 01:27:58 GMT',
   'content-type': 'application/json',
   'content-length': '684',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2a3ee44f-522c-4838-b015-6fb1057a20f1'},
  'RetryAttempts': 0},
 'itemList': [{'itemId': '-8669770849634489893', 'score': 0.0137572},
  {'itemId': '-5570129644089964821', 'score': 0.0111488},
  {'itemId': '-6843047699859121724', 'score': 0.0085601},
  {'itemId': '358192030955601318', 'score': 0.0079786},
  {'itemId': '-8618420761918493321', 'score': 0.0068266}],
 'recommendationId': 'RID-64-421a-b5b8-3d2e161a77cb-CID-482b34'}

In [28]:
json_array = []
json_record = {
    "itemId": 'madeupitem222', 
    "properties": f"""{{"creationTimestamp": 1714006143, "TRAINING_TEXT": "test new item", "LANG": "en", "ARTICLE_GENRE": "fake article"}}"""
}
json_array.append(json_record)

In [29]:
response = personalize_events.put_items(
    datasetArn=workshop_items_dataset_arn,
    items=json_array
)
print(response)

{'ResponseMetadata': {'RequestId': '0f165b43-26af-414f-a51c-886876d8c04a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 29 May 2024 01:28:01 GMT', 'content-type': 'application/json', 'content-length': '0', 'connection': 'keep-alive', 'x-amzn-requestid': '0f165b43-26af-414f-a51c-886876d8c04a'}, 'RetryAttempts': 0}}


In [49]:
response = personalize_events.put_events(
    trackingId="4ea6be84-eb39-4b11-8f59-b76609ac56df",
    userId='madeup-user',
    sessionId='string',
    eventList=[
        {
            'eventId': 'madeupevent222',
            'eventType': 'VIEW',
            'itemId': 'madeupitem222',
            'sentAt': 1714006143,
        },
    ]
)